In [ ]:
import datetime
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

import plotly.express as px
from dash import Dash, dcc, html, Input, Output, State
from jupyter_dash import JupyterDash

import sys
sys.path.append('./rtaUtils')
from rtaUtils import data_loading, common, sort_vectors, paths


config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'newplot',
    'height': 1000,
    'width': 1000,
    'scale':1 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

airport_list = ('LEBL','LPPT','LFPO','LFPG','EGLL','LPPR','EDDF','EBBR','EHAM','LEBB','LIRF',
                'EDDM','EGKK','LEVC','LSGG','LECO','LIMC','EIDW','LFML','LEZL','LOWW','LIPZ',
                'LIPE','LFLL','EDDL','EDDB','LTFM','LFMN','LFRS','LROP','EDDP','LGAV','EDDH',
                'LHBP','EKCH','EGCC','ELLX','LKPR','LIRN','LBSF','EPWA')

## Inicialización

In [ ]:
# Estilos de los ejemplos de Dash
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [ ]:
# Carga de datos
data = pd.read_csv('./sort_stats.csv').drop_duplicates(subset='fpId', keep='last')
data['flightDate'] = pd.to_datetime(data.flightDate)

## Estructura

In [ ]:
dates = common.get_dates_between(date_start = data.flightDate.min().strftime('%Y-%m-%d'), 
                                 date_end   = data.flightDate.max().strftime('%Y-%m-%d'))

dates = [f'2022-{m:0>2}-{d:0>2}' for m in range(1,10) for d in range(1,32) 
         if f'2022-{m:0>2}-{d:0>2}' not in data.flightDate.dt.date.astype(str).values]

# Styles
style_selector = dict(marginLeft=20)

# Input elements
date_selector = dcc.DatePickerSingle(
    id = 'dateSelector',
    placeholder = 'Select one...',
    date=data.flightDate.min(),
    initial_visible_month=data.flightDate.min(),
    min_date_allowed = data.flightDate.min(),
    max_date_allowed = data.flightDate.max(),
    display_format='DD/MM/YYYY',
    disabled_days=dates
)

airport_selector = dcc.Dropdown(
    id = 'airportSelector',
    options = [dict(label=x, value=x) for x in sorted(airport_list)],
    placeholder='Select one...',
    multi=True,
)

trajectory_selector = dcc.Dropdown(
    id = 'trajectorySelector',
    placeholder='Select one...',
    multi=False,
)

distance_slider = dcc.RangeSlider(
    id='distanceSlider', 
    min=0, max=100, step=25, value=[0,100], 
    pushable=5, allowCross=False,
    tooltip=dict(placement='top', always_visible=False),
)

value_selector = dcc.Dropdown(
    id='valueSelector', 
    value='ordenInicial', 
    clearable=False, 
    options=dict(ordenInicial='Orden inicial', track='Track', RTA='RTA')
)

In [ ]:
# Layout
app.layout = html.Div(children=[
    html.Div(style=dict(width='100%', borderWidth=1, display='flex', borderStyle='solid', borderRadius=10), children=[
        html.Div(style=dict(width='30%', margin=10), children=[
            html.Div(style=dict(display='flex'), children=[
                html.Label(style=dict(width='75%'), 
                           children=['Trajectory', trajectory_selector]),
                html.Label(style=dict(width='25%', marginLeft=10), 
                           children=['Feature:', value_selector]),
            ]),
            html.Label(style=dict(marginTop=5),
                       children=['Distance', distance_slider]),
        ]),
        html.Div(style=dict(width='15%', margin=10, marginLeft=20), children=[
            html.Label('Flight date', className='d-table'), 
            date_selector,
            html.Label('Origin airport', className='d-table', style=dict(marginTop=5)),
            airport_selector,
        ]),
        html.Div(id='trayectoryDescription', style=dict(width='50%', margin=10))
        
    ]),
    html.Div(style=dict(width='100%', display='flex', marginTop=10), children=[
        html.Div(style=dict(width='50%'), children=[
            dcc.Graph(id='grafico_inicial', style=dict(width='100%'))
        ]),
        html.Div(style=dict(width='50%'), children=[ 
            dcc.Graph(id='grafico_reordenado', style=dict(width='100%'))
        ])
    ])
])

## Callbacks

In [ ]:
@app.callback(
    [Output(component_id='trajectorySelector', component_property='options')],
    [Input(component_id='dateSelector', component_property='date'),
     Input(component_id='airportSelector', component_property='value')]
)
def filtrar_trayectorias(fecha, origen):
    df = pd.read_parquet(paths.sorted_data_path / f'{fecha[:10].replace("-","")}.parquet')
    if origen:
        df = df[df.aerodromeOfDeparture.isin(origen)]  

    labels = pd.merge(data[['fpId','rotation','ratio','oscillation']].drop_duplicates(),
                      df[['fpId','aerodromeOfDeparture']].drop_duplicates(), on='fpId')
    labels = ([{'label': f'{x[1].aerodromeOfDeparture} {x[1].fpId}   ({x[1].rotation:>4.0f}|{x[1].ratio:>5.2f}%)   {x[1].oscillation*100:>3.0f}%', 
                'value': x[1].fpId} 
             for x in labels.sort_values(['aerodromeOfDeparture','ratio']).iterrows()],)
    return labels


@app.callback(
    [Output(component_id='distanceSlider', component_property='marks'),
     Output(component_id='distanceSlider', component_property='max'),
     Output(component_id='distanceSlider', component_property='value')],
    [Input(component_id='trajectorySelector', component_property='value')],
    [State(component_id='dateSelector', component_property='date')]
)
def actualizar_slider(trajectory, fecha):
    if trajectory is None:
        marks = {0:'0',100:'100'}
        max_value = 100
    else:
        df = pd.read_parquet(paths.sorted_data_path / f'{fecha[:10].replace("-","")}.parquet')
        df = df[df.fpId == trajectory]
        marks = {x:str(x) for x in range(0,int(df.distance_dst.max()+100),100)}
        max_value = max(marks)
    value = [0, max_value]
    
    return (marks, max_value, value)


@app.callback(
    [Output(component_id='grafico_inicial', component_property='figure'),
     Output(component_id='grafico_reordenado', component_property='figure'),
     Output(component_id='trayectoryDescription', component_property='children')],
    [Input(component_id='trajectorySelector', component_property='value'),
     Input(component_id='distanceSlider', component_property='value'),
     Input(component_id='valueSelector', component_property='value')],
    [State(component_id='dateSelector', component_property='date')]
)
def actualizar_grafico_perfiles(trajectory, rango, color_scale, fecha):
    df = pd.read_parquet(paths.sorted_data_path / f'{fecha[:10].replace("-","")}.parquet')
    df = df[df.fpId == trajectory]
    max_distance = df.distance_dst.max()
    df = df[df.distance_dst.between(rango[0],rango[1])]
    
    fig_inicial = px.scatter_mapbox(
        df, lat='latitude', lon='longitude',
        height=725, zoom=4, color=color_scale,
        mapbox_style='open-street-map',
        hover_data=['ordenInicial','ordenFinal','track','distance_org', 'distance_dst'], 
        title='Initial order' if color_scale=='ordenInicial' else 'Track values'
     )
    
    fig_reordenado= px.scatter_mapbox(
        df, lat='latitude', lon='longitude',
        height=725, zoom=4, color='ordenFinal',
        mapbox_style='open-street-map', 
        hover_data=['ordenInicial','ordenFinal','track','ground', 
                    'distance_org','distance_dst'], 
        title='Final order'
     )
    margins = dict(l=5, r=5, b=100, t=40, pad=0 )
    fig_inicial.update_layout(coloraxis=dict(colorbar=dict(orientation='h', y=-0.12)), margin=margins)
    fig_reordenado.update_layout(coloraxis=dict(colorbar=dict(orientation='h', y=-0.12)), margin=margins)
    
    if trajectory is None:
        contenido = []
        
    else:
        tray_data = data[data.fpId==trajectory]
        contenido = (
            html.Table(style=dict(marginLeft=20), children=[
                html.Tbody(children=[
                    html.Tr(children=[
                        html.Td('Trajectory:'), html.Td(trajectory),
                        html.Td('Origin:'), html.Td(df.iloc[0].aerodromeOfDeparture),
                        html.Td('Initial distance:'), html.Td(f'{tray_data.initial.values[0]:>.2f}'),
                        html.Td('Final distance:'), html.Td(f'{tray_data.final.values[0]:>.2f}'),
                    ]),
                    html.Tr(children=[
                        html.Td('Rotation:'), html.Td(f'{tray_data.rotation.values[0]:>.0f}'),
                        html.Td('Oscillation:'), html.Td(f'{tray_data.oscillation.values[0]:>.0f}%'),
                        html.Td('Oscillation sample:'), html.Td(f'{tray_data.oscillation_sample.values[0]:>.0f}'),
                        html.Td('Distance variation:'), html.Td(f'{tray_data.ratio.values[0]:>+.2f}%'),
                    ]),
                ])
            ]))
    
    return (fig_inicial, 
            fig_reordenado, 
            contenido)

In [ ]:
app.run_server(debug = True)
# mode='inline'